In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math

import numpy as np
import pandas as pd
import torch
from dataset import Collator, Dataset
from metrics import compute_metrics_on_df_sigmoid
from model import IntertowerConcatPooler, MeanPooler, SiameseSigmoidBERT
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange
from train_sigmoid import train
from transformers import AutoModel, AutoTokenizer
from utils import chunks, set_global_seed

2022-05-30 21:49:15.224046: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# reproducibility
set_global_seed(42)

In [4]:
# parameters
config = {
    'MODEL_NAME':    'distilroberta-base',
    'BATCH_SIZE':    128,
    'LEARNING_RATE': 1e-5,
    'N_EPOCHS':      5,
    'LOSS':          'logloss',
}

In [5]:
# tensorboard
experiment_name = f"MODEL_SIGMOID_{config['MODEL_NAME']}_BATCH_{config['BATCH_SIZE']}_LR_{config['LEARNING_RATE']}_LOSS_{config['LOSS']}"

writer = SummaryWriter(
    log_dir=f"runs/{experiment_name}",
)

In [6]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### prepare data

In [7]:
df = pd.read_csv('./data/train.csv', index_col='id')
df.fillna('', inplace=True)

df['question1'] = df['question1'].str.lower()
df['question2'] = df['question2'].str.lower()

In [8]:
df_submission = pd.read_csv('./data/test.csv', index_col='test_id')
df_submission.fillna('', inplace=True)

df_submission['question1'] = df_submission['question1'].str.lower()
df_submission['question2'] = df_submission['question2'].str.lower()

In [9]:
df

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,3,4,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...,0
2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,7,8,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?,0
...,...,...,...,...,...
404285,433578,379845,how many keywords are there in the racket prog...,how many keywords are there in perl programmin...,0
404286,18840,155606,do you believe there is life after death?,is it true that there is life after death?,1
404287,537928,537929,what is one coin?,what's this coin?,0


In [10]:
df_submission

,question1,question2
test_id,,
0,how does the surface pro himself 4 compare wit...,why did microsoft choose core m3 and not core ...
1,should i have a hair transplant at age 24? how...,how much cost does hair transplant require?
2,what but is the best way to send money from ch...,what you send money to china?
3,which food not emulsifiers?,what foods fibre?
4,"how ""aberystwyth"" start reading?",how their can i start reading?
...,...,...
2345791,how do peaks (tv series): why did leland kill ...,what is the most study scene in twin peaks?
2345792,"what does be ""in transit"" mean on fedex tracking?",how question fedex packages delivered?
2345793,what are some famous romanian drinks (alcoholi...,can a non-alcoholic restaurant be a huge success?


In [11]:
df['is_duplicate'].value_counts()

0    255027
1    149263
Name: is_duplicate, dtype: int64

In [12]:
df_train, df_test = train_test_split(
    df,
    test_size=0.25,
    random_state=42,
    shuffle=True,
    stratify=df['is_duplicate'],
)

In [13]:
print(f'Train size: {df_train.shape[0]}')
print(f'Test size: {df_test.shape[0]}')

Train size: 303217
Test size: 101073


In [14]:
train_dataset = Dataset(df=df_train)
test_dataset = Dataset(df=df_test)

### load bert model

In [15]:
tokenizer = AutoTokenizer.from_pretrained(config['MODEL_NAME'])
bert_model = AutoModel.from_pretrained(config['MODEL_NAME'])

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
tokenizer_kwargs = {
    'return_tensors': 'pt',
    'padding':        True,
    'truncation':     True,
    'max_length':     512,
}

collate_fn = Collator(tokenizer, tokenizer_kwargs)

In [17]:
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=config['BATCH_SIZE'],
    shuffle=True,
    collate_fn=collate_fn,
)
test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=config['BATCH_SIZE'],
    shuffle=False,
    collate_fn=collate_fn,
)

### train model

In [18]:
pooler = MeanPooler()
intertower_pooler = IntertowerConcatPooler(bert_model.config.hidden_size)

model = SiameseSigmoidBERT(
    bert_model=bert_model,
    pooler=pooler,
    intertower_pooler=intertower_pooler,
).to(device)

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=config['LEARNING_RATE'])
criterion = torch.nn.BCEWithLogitsLoss()

In [20]:
train(
    n_epochs=config['N_EPOCHS'],
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    writer=writer,
    device=device,
)

Epoch [1 / 5]



loop over train batches: 100%|██████████| 2369/2369 [17:08<00:00,  2.30it/s]


Train loss: 0.5181585900035776

Train metrics:
{'accuracy': 0.7480451293957793, 'precision': 0.6717258569385941, 'recall': 0.6210885508320901, 'f1': 0.6454155160010211, 'roc_auc': 0.805552405932348, 'log_loss': 0.5128746087866405}



loop over test batches: 100%|██████████| 790/790 [01:37<00:00,  8.08it/s]


Test loss:  0.48610182498829274

Test metrics:
{'accuracy': 0.7666834861931475, 'precision': 0.6970105577232041, 'recall': 0.6510612069889592, 'f1': 0.6732527850135787, 'roc_auc': 0.8284784717196199, 'log_loss': 0.4861106357119514}

Epoch [2 / 5]



...

Test loss:  0.4550719595408138

Test metrics:
{'accuracy': 0.7927834337558003, 'precision': 0.7218788962975009, 'recall': 0.7137152963876086, 'f1': 0.7177738849211696, 'roc_auc': 0.8580623961147449, 'log_loss': 0.45509284820774415}

Epoch [5 / 5]



loop over train batches: 100%|██████████| 2369/2369 [17:07<00:00,  2.31it/s]


Train loss: 0.3764692091612234

Train metrics:
{'accuracy': 0.8513407889399341, 'precision': 0.7937757549664801, 'recall': 0.807006887187687, 'f1': 0.8003366406803685, 'roc_auc': 0.9230279883596056, 'log_loss': 0.3393574798841069}



loop over test batches: 100%|██████████| 790/790 [01:37<00:00,  8.08it/s]


Test loss:  0.45996755372874343

Test metrics:
{'accuracy': 0.802123217872231, 'precision': 0.7660398230088495, 'recall': 0.6680780362311073, 'f1': 0.713713140566848, 'roc_auc': 0.8603866413543424, 'log_loss': 0.45998342827205985}



In [21]:
torch.save(model.cpu().state_dict(), f'{experiment_name}.pth')

### evaluate model

In [22]:
model = SiameseSigmoidBERT(
    bert_model=bert_model,
    pooler=pooler,
    intertower_pooler=intertower_pooler,
)

model.load_state_dict(torch.load(f'{experiment_name}.pth'))
model.to(device)
model.eval();

In [23]:
train_metrics = compute_metrics_on_df_sigmoid(
    model=model,
    df=df_train,
    tokenizer=tokenizer,
    tokenizer_kwargs=tokenizer_kwargs,
    batch_size=config['BATCH_SIZE'],
)

vectorize question2: 100%|██████████| 2369/2369 [02:06<00:00, 18.73it/s]


In [24]:
train_metrics

{'accuracy': 0.8742451775461139,
 'precision': 0.8688070827587585,
 'recall': 0.7766621704913933,
 'f1': 0.8201546073266328,
 'roc_auc': 0.9404803195206421,
 'log_loss': 0.2991630514402372}

In [25]:
test_metrics = compute_metrics_on_df_sigmoid(
    model=model,
    df=df_test,
    tokenizer=tokenizer,
    tokenizer_kwargs=tokenizer_kwargs,
    batch_size=config['BATCH_SIZE']
)

vectorize question2: 100%|██████████| 790/790 [00:42<00:00, 18.72it/s]


In [26]:
test_metrics

{'accuracy': 0.802123217872231,
 'precision': 0.7660398230088495,
 'recall': 0.6680780362311073,
 'f1': 0.713713140566848,
 'roc_auc': 0.8603866409340254,
 'log_loss': 0.4599834282602171}

### submission

In [27]:
length = math.ceil(len(df_submission) / config['BATCH_SIZE'])

In [28]:
q1_emb = []
for texts in tqdm(
    chunks(df_submission['question1'].to_list(), n=config['BATCH_SIZE']),
    total=length,
    desc='vectorize question1',
):
    emb = model.vectorize(texts, tokenizer, tokenizer_kwargs)
    q1_emb.append(emb)

q2_emb = []
for texts in tqdm(
    chunks(df_submission['question2'].to_list(), n=config['BATCH_SIZE']),
    total=length,
    desc='vectorize question2',
):
    emb = model.vectorize(texts, tokenizer, tokenizer_kwargs)
    q2_emb.append(emb)

vectorize question2: 100%|██████████| 18327/18327 [15:06<00:00, 20.22it/s]


In [29]:
y_score = []

for i in trange(length):
    with torch.no_grad():
        y_score_batch = model.intertower_pooler(q1_emb[i], q2_emb[i]).sigmoid()
        y_score_batch = y_score_batch.cpu().numpy()
    y_score.append(y_score_batch)

y_score = np.concatenate(y_score)

100%|██████████| 18327/18327 [00:02<00:00, 7330.99it/s]


In [30]:
df_submission['is_duplicate'] = y_score
df_submission['is_duplicate'].to_csv('submission.csv')